In [1]:
import numpy as np
import cv2
import glob
import os

In [4]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
left_imgpoints = [] # 2d points in image plane.
right_imgpoints = [] # 2d points in image plane.

left_images = glob.glob('../img/left/*.jpg')
right_images = glob.glob('../img/right/*.jpg')

for left_fname, right_fname in zip(left_images, right_images):
    left_img = cv2.imread(left_fname)
    right_img = cv2.imread(right_fname)
    left_gray = cv2.cvtColor(left_img,cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img,cv2.COLOR_BGR2GRAY)
    left_ret, left_corners = cv2.findChessboardCorners(left_gray, (7,6), None)
    right_ret, right_corners = cv2.findChessboardCorners(right_gray, (7,6), None)
    if left_ret and right_ret:
        objpoints.append(objp)
        cv2.cornerSubPix(left_gray, left_corners, (11,11), (-1,-1), criteria)
        cv2.cornerSubPix(right_gray, right_corners, (11,11), (-1,-1), criteria)
        left_imgpoints.append(left_corners)
        right_imgpoints.append(right_corners)
        
imgSize = left_gray.shape[::-1];        
ret1, mtx1, dist1, rvecs1, tvecs1 = cv2.calibrateCamera(objpoints, left_imgpoints, imgSize, None, None)
ret2, mtx2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(objpoints, right_imgpoints, imgSize, None, None)
retv, mtx1, dist1, mtx2, dist2, R, T, E, F = \
cv2.stereoCalibrate(objpoints, left_imgpoints, right_imgpoints, mtx1, dist1, mtx2, dist2, imgSize, criteria)

In [5]:
mtx1

array([[ 535.04584846,    0.        ,  341.29707287],
       [   0.        ,  535.34171797,  233.71686258],
       [   0.        ,    0.        ,    1.        ]])

In [8]:
if not os.path.isdir('../img/left_undistort'):
    os.makedirs('../img/left_undistort')
if not os.path.isdir('../img/right_undistort'):
    os.makedirs('../img/right_undistort')
for fname in left_images + right_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = img.shape[:2]
    if fname[2] == 'l':
        mtx = mtx1
        dist = dist1
        file_name = '../img/left_undistort/'
    elif fname[2] == 'r':
        mtx = mtx2
        dist = dist2
        file_name = '../img/right_undistort/'
#     newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, None)
    # crop the image
#     x,y,w,h = roi
#     dst = dst[y:y+h, x:x+w]
    path = file_name+fname.split('/')[-1]
    cv2.imwrite(path, dst)

NameError: name 'mtx' is not defined

In [148]:
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtx1, dist1, mtx2, dist2, imgSize, R, T)

In [149]:
mapL1, mapL2 = cv2.initUndistortRectifyMap(mtx1, dist1, R1, P1, imgSize, cv2.CV_32FC1);
mapR1, mapR2 = cv2.initUndistortRectifyMap(mtx2, dist2, R2, P2, imgSize, cv2.CV_32FC1);

In [151]:
pic_num = "01"

if not os.path.isdir('./rectify'):
    os.makedirs('./rectify')
img_left = cv2.imread("./left_undistort/left"+pic_num+".jpg")
gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
img_right = cv2.imread("./right_undistort/right"+pic_num+".jpg")
gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

cv2.imwrite("./rectify/org_left"+pic_num+".jpg", gray_left)
cv2.imwrite("./rectify/org_right"+pic_num+".jpg", gray_right)

undistorted_rectified_left = cv2.remap(gray_left, mapL1, mapL2, cv2.INTER_LINEAR);
undistorted_rectified_right = cv2.remap(gray_right, mapL1, mapL2, cv2.INTER_LINEAR);

# crop the image
# x, y, w, h = roi1
# undistorted_rectified_left = undistorted_rectified_left[y:y+h, x:x+w]
# x, y, w, h = roi2
# undistorted_rectified_right = undistorted_rectified_right[y:y+h, x:x+w]

cv2.imwrite("./rectify/left"+pic_num+".jpg", undistorted_rectified_left)
cv2.imwrite("./rectify/right"+pic_num+".jpg", undistorted_rectified_right)

True